# Treino/Teste modelo

In [1]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
import regex as re
import string

## 1. Importando dados da preparação e stopwords

In [2]:
PARQUET_MPO_PREPARADOS = '../../Data/Processed/mpo_preparados.parquet'

df_mpo = pd.read_parquet(PARQUET_MPO_PREPARADOS)

In [3]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/marcelo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df_mpo.head(5)

,ID,Level,MpoIdentificador,MpoAreaEletrica,MpoCentro,MpoLigadoDesligado,Texto,Título
0,161279,1,IO-PM.6MD,6MD,CNOS,Não definido,Estabelecer os procedimentos a serem seguido...,OBJETIVO
1,161280,2,IO-PM.6MD,6MD,CNOS,Não definido,2.1. Os procedimentos contidos ne...,CONSIDERAÇÕES GERAIS
2,161281,3,IO-PM.6MD,6MD,CNOS,Não definido,1. OBJETIVO 2. ............................,PREPARAÇÃO PARA MANOBRAS EM LINHAS DE TRANSMISSÃO
3,161282,3.1,IO-PM.6MD,6MD,CNOS,Não definido,1. OBJETIVO 2. ............................,LT 230 kV Coletora Porto Velho / Porto Velho C...
4,161283,3.1.1,IO-PM.6MD,6MD,CNOS,Desligar,3.1.1.1. Desenergização da LT 230 ...,Desenergização da LT 230 kV Coletora Porto Vel...


## 2. Vetorizando com tf-idf

In [5]:
#vectorizer_tfidf = TfidfVectorizer(stop_words=stopwords) # , ngram_range = (1, 2)
#X = vectorizer_tfidf.fit_transform(df_mpo['Texto_Limpo'])
vectorizer_tfidf = TfidfVectorizer(stop_words=stopwords, min_df = 0.001, max_df = 0.99) # , ngram_range = (1, 2)
X = vectorizer_tfidf.fit_transform(df_mpo[(df_mpo['MpoLigadoDesligado']=='Desligar') | (df_mpo['MpoLigadoDesligado']=='Ligar')]['Texto'])
vocab = np.array(vectorizer_tfidf.get_feature_names())

In [6]:
#1st run - <5456x8001 sparse matrix of type '<class 'numpy.float64'>'
#	with 306816 stored elements in Compressed Sparse Row format>

#2nd run - 0.05 - 0.95 - <5456x208 sparse matrix of type '<class 'numpy.float64'>'
#	with 170520 stored elements in Compressed Sparse Row format>


#3rd run - 0.01 - 0.99 - <5456x921 sparse matrix of type '<class 'numpy.float64'>'
#	with 263489 stored elements in Compressed Sparse Row format>

X

<3491x2564 sparse matrix of type '<class 'numpy.float64'>'
	with 151516 stored elements in Compressed Sparse Row format>

In [7]:
vocab[2000:2030]

array(['proceder', 'procedimento', 'procedimentos', 'procurar',
       'programa', 'programadas', 'programado', 'programados', 'proibida',
       'proibitivo', 'promissão', 'proprietária', 'proprietário',
       'proteção', 'proveniente', 'providenciar', 'providencie',
       'providências', 'provoca', 'provocar', 'provoque', 'prs', 'pré',
       'prévio', 'próxima', 'próximo', 'pss', 'pte', 'pólo', 'q1'],
      dtype='<U19')

## 4. Modelagem de tópicos com NMF

In [8]:
def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [9]:
from sklearn.decomposition import NMF

#d = X.shape[0]
num_top_words=6

model_nmf = NMF(n_components=100, init='random', random_state=0)

W1 = model_nmf.fit_transform(X)
H1 = model_nmf.components_


print(show_topics(H1))

['lt energização desta sentido tensão partir', 'operador fechamento autonomia oi situações demais', '69 transformador kv 13 230 mva', 'específica lado cemig energização instalação io', '500 kv 550 300 terminal reator', 'iguaçu nova angra taubaté grajaú faixas', 'c2 c1 lt terminal escolhido c3', 'biguaçu palhoça eletrosul florianópolis gaspar ocorrer', '525 itá disjuntores eletrosul kv areia', 'autotransformador lado 13 230 kv ltc', '138 kv lado transformador mva defasador', 'curitiba leste cidade uberaba get copel', 'iniciando instruções instalações envolvidas energização inicia', 'ute viana goiânia suzano araucária coe', 'dutra presidente imperatriz açailândia eletronorte oeorm', 'campo comprido get mourão copel umbará', 'cron ne chesf banabuiú milagres picos', 'ii c1 luís miranda sobral kv', 'nova mutum ponte brasnorte sorriso avanhandava', 'tf 13 transformador lado 230 disjuntor', '345 kv 362 itutinga pimenta viana', 'neves paulino elevadora sete lagoas steag', '01 02 03 nco eletron

In [10]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=8, init='k-means++', max_iter=100, n_init=10, verbose=True)
km.fit_transform(X)

print(show_topics(km.cluster_centers_))

Initialization complete
Iteration 0, inertia 5378.009710694339
Iteration 1, inertia 3026.7540767715686
Iteration 2, inertia 2979.0662777153625
Iteration 3, inertia 2962.6998385876464
Iteration 4, inertia 2954.492054995869
Iteration 5, inertia 2951.34731739851
Iteration 6, inertia 2949.174059020434
Iteration 7, inertia 2948.3244375576187
Iteration 8, inertia 2948.0560090715267
Iteration 9, inertia 2947.9529365030935
Iteration 10, inertia 2947.9406586179657
Iteration 11, inertia 2947.9364650717735
Iteration 12, inertia 2947.9292335584564
Converged at iteration 12: strict convergence.
Initialization complete
Iteration 0, inertia 5469.24542343453
Iteration 1, inertia 3029.5054389234947
Iteration 2, inertia 2975.656437579
Iteration 3, inertia 2957.7103023792656
Iteration 4, inertia 2949.6493446888408
Iteration 5, inertia 2941.901293421595
Iteration 6, inertia 2940.422634418781
Iteration 7, inertia 2939.568347496692
Iteration 8, inertia 2938.5289138534076
Iteration 9, inertia 2936.8283981278

In [11]:
from collections import Counter
c = Counter(km.labels_)
c

Counter({4: 419, 5: 1114, 1: 529, 0: 191, 3: 740, 6: 209, 7: 140, 2: 149})

In [17]:
y = km.labels_
print(Counter(y))

Counter({5: 1114, 3: 740, 1: 529, 4: 419, 6: 209, 0: 191, 2: 149, 7: 140})


In [15]:
cc = km.cluster_centers_
cc

array([[0.00000000e+00, 1.00737823e-03, 0.00000000e+00, ...,
        9.89290095e-04, 0.00000000e+00, 1.23995897e-03],
       [5.89885591e-05, 1.48207960e-03, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [5.31894463e-04, 3.23227205e-03, 0.00000000e+00, ...,
        6.75784240e-04, 6.71297031e-04, 6.83738202e-05],
       [0.00000000e+00, 2.21048729e-03, 2.88799933e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])